# listm

In [ ]:
// // test

open testing

prototype append t : t -> t -> t

()



## append

In [ ]:
instance append list t =
    listm.append

()



## /@

In [ ]:
inl (/@) a b =
    b |> append a

()



In [ ]:
// // test

[ "a"; "b" ] /@ [ "c"; "d" ]
|> _equal [ "a"; "b"; "c"; "d" ]

type UH0 =
    | UH0_0 of string * UH0
    | UH0_1
let rec method0 () : unit =
    let v0 : string = "a"
    let v1 : string = "b"
    let v2 : string = "c"
    let v3 : string = "d"
    let v4 : UH0 = UH0_1
    let v5 : UH0 = UH0_0(v3, v4)
    let v6 : UH0 = UH0_0(v2, v5)
    let v7 : UH0 = UH0_0(v1, v6)
    let v8 : UH0 = UH0_0(v0, v7)
    let v9 : UH0 = UH0_1
    let v10 : UH0 = UH0_0(v3, v9)
    let v11 : UH0 = UH0_0(v2, v10)
    let v12 : UH0 = UH0_0(v1, v11)
    let v13 : UH0 = UH0_0(v0, v12)
    let v14 : string = $"__expect / actual: %A{v8} / expected: %A{v13}"
    ()
method0()



## init_series

In [ ]:
inl init_series start end inc =
    inl total : f64 = conv ((end - start) / inc) + 1
    listm.init total (conv >> (*) inc >> (+) start)

()



In [ ]:
// // test

init_series 0 1 0.5
|> _equal [ 0f64; 0.5; 1 ]

type UH0 =
    | UH0_0 of float * UH0
    | UH0_1
let rec method0 () : unit =
    let v0 : UH0 = UH0_1
    let v1 : UH0 = UH0_0(1.0, v0)
    let v2 : UH0 = UH0_0(0.5, v1)
    let v3 : UH0 = UH0_0(0.0, v2)
    let v4 : UH0 = UH0_1
    let v5 : UH0 = UH0_0(1.0, v4)
    let v6 : UH0 = UH0_0(0.5, v5)
    let v7 : UH0 = UH0_0(0.0, v6)
    let v8 : string = $"__expect / actual: %A{v3} / expected: %A{v7}"
    ()
method0()



## try_item

In [ ]:
inl rec try_item i = function
    | Cons (x, _) when i = 0 => Some x
    | Cons (_, xs) => try_item (i - 1) xs
    | Nil => None

()



In [ ]:
// // test

listm.init 10i32 id
|> try_item 9i32
|> _equal (Some 9)

type [<Struct>] US0 =
    | US0_0
    | US0_1 of f1_0 : int32
let rec method0 () : unit =
    let v0 : US0 = US0_1(9)
    let v1 : US0 = US0_1(9)
    let v2 : string = $"__expect / actual: %A{v0} / expected: %A{v1}"
    ()
method0()



In [ ]:
// // test

listm.init 10i32 id
|> try_item 10i32
|> _equal None

type [<Struct>] US0 =
    | US0_0
    | US0_1 of f1_0 : int32
let rec method0 () : unit =
    let v0 : US0 = US0_0
    let v1 : US0 = US0_0
    let v2 : string = $"__expect / actual: %A{v0} / expected: %A{v1}"
    ()
method0()



## list_item

In [ ]:
inl item i =
    try_item i >> optionm.value

()



In [ ]:
// // test

listm.init 10i32 id
|> item 9i32
|> _equal 9

let rec method0 () : unit =
    let v0 : string = $"__expect / actual: %A{9} / expected: %A{9}"
    ()
method0()



In [ ]:
// // test

fun () =>
    listm.init 10i32 id
    |> item 10i32
    |> ignore
|> _throws
|> _equal (Some "Option does not have a value.")

type [<Struct>] US0 =
    | US0_0
    | US0_1 of f1_0 : string
let rec closure0 () () : unit =
    let v0 : int32 = failwith<int32> "Option does not have a value."
    ()
and closure1 () (v0 : string) : US0 =
    US0_1(v0)
and method0 () : unit =
    let v0 : (unit -> unit) = closure0()
    let v1 : US0 = US0_0
    let v2 : (string -> US0) = closure1()
    let v3 : US0 = try v0 (); v1 with ex -> v2 ex.Message
    let v4 : string = "Option does not have a value."
    let v5 : US0 = US0_1(v4)
    let v6 : string = $"__expect / actual: %A{v3} / expected: %A{v5}"
    let v10 : bool =
        match v3 with
        | US0_1(v8) -> (* Some *)
            let v9 : bool = v8 = "Option does not have a value."
            v9
        | _ ->
            false
    let v11 : bool = v10 = false
    if v11 then
        failwith<unit> v6
method0()



## try_item_

In [ ]:
let rec try_item_ i = function
    | Cons (x, _) when i = 0 => Some x
    | Cons (_, xs) => try_item_ (i - 1) xs
    | Nil => None

()



## item_

In [ ]:
inl item_ i =
    try_item_ i >> optionm.value

()



## sum

In [ ]:
inl sum list =
    list |> listm.fold (+) 0

()



In [ ]:
// // test

listm.init 10i32 id
|> sum
|> _equal 45

let rec method0 () : unit =
    let v0 : string = $"__expect / actual: %A{45} / expected: %A{45}"
    ()
method0()



## unzip

In [ ]:
inl unzip list =
    (([], []), list)
    ||> listm.fold fun (acc_x, acc_y) (x, y) =>
        x :: acc_x, y :: acc_y
    |> fun x, y =>
        x |> listm.rev, y |> listm.rev

()



In [ ]:
// // test

listm.init 10i32 id
|> listm.map (fun x => x, x)
|> unzip
|> fun x, y =>
    x |> sum
    |> _equal 45

    y |> sum
    |> _equal 45

let rec method0 () : unit =
    let v0 : string = $"__expect / actual: %A{45} / expected: %A{45}"
    let v1 : string = $"__expect / actual: %A{45} / expected: %A{45}"
    ()
method0()



## try_index_of

In [ ]:
inl try_index_of item list =
    inl rec loop i = function
        | [] => None
        | x :: xs =>
            if x = item
            then Some i
            else loop (i + 1) xs
    loop 0 list

inl index_of item =
    try_index_of item >> optionm.value

inl try_index_of_ item list =
    let rec loop i = function
        | [] => None
        | x :: xs =>
            if x = item
            then Some i
            else loop (i + 1) xs
    loop 0 list

inl index_of_ item =
    try_index_of_ item >> optionm.value

inl try_index_of__ item list =
    inl i = mut 0
    inl list = mut list
    inl result = mut None
    let rec loop () =
        match *list with
        | [] => result <- None
        | x :: xs =>
            if x = item
            then result <- Some *i
            else
                i <- *i + 1
                list <- xs
                loop ()
    loop ()
    *result

inl index_of__ item =
    try_index_of__ item >> optionm.value

()



In [ ]:
// // test

listm.init 10i32 id
|> index_of 5i32
|> _equal 5i32

let rec method0 () : unit =
    let v0 : string = $"__expect / actual: %A{5} / expected: %A{5}"
    ()
method0()



In [ ]:
// // test

listm.init 10i32 id
|> try_index_of 10i32
|> _equal (None : option i32)

type [<Struct>] US0 =
    | US0_0
    | US0_1 of f1_0 : int32
let rec method0 () : unit =
    let v0 : US0 = US0_0
    let v1 : US0 = US0_0
    let v2 : string = $"__expect / actual: %A{v0} / expected: %A{v1}"
    ()
method0()



## try_find

In [ ]:
inl try_find fn list =
    inl rec loop = function
        | [] => None
        | x :: xs =>
            if fn x
            then Some x
            else loop xs
    loop list

inl try_find_ fn list =
    let rec loop = function
        | [] => None
        | x :: xs =>
            if fn x
            then Some x
            else loop xs
    loop list

()



In [ ]:
// // test

listm.init 10i32 id
|> try_find ((=) 5i32)
|> _equal (Some 5i32)

type [<Struct>] US0 =
    | US0_0
    | US0_1 of f1_0 : int32
let rec method0 () : unit =
    let v0 : US0 = US0_1(5)
    let v1 : US0 = US0_1(5)
    let v2 : string = $"__expect / actual: %A{v0} / expected: %A{v1}"
    ()
method0()



In [ ]:
inl find x =
    try_find x >> optionm.value

inl find_ x =
    try_find_ x >> optionm.value

()



In [ ]:
// // test

listm.init 10i32 id
|> find ((=) 5i32)
|> _equal 5i32

let rec method0 () : unit =
    let v0 : string = $"__expect / actual: %A{5} / expected: %A{5}"
    ()
method0()



## choose

In [ ]:
inl choose f l =
    (l, [])
    ||> listm.foldBack fun x acc =>
        match f x with
        | Some y => y :: acc
        | None => acc

()



In [ ]:
// // test

listm.init 10i32 id
|> choose (fun x => if x % 2 = 0 then Some x else None)
|> _equal [ 0; 2; 4; 6; 8 ]

type UH0 =
    | UH0_0 of int32 * UH0
    | UH0_1
let rec method0 () : unit =
    let v0 : UH0 = UH0_1
    let v1 : UH0 = UH0_0(8, v0)
    let v2 : UH0 = UH0_0(6, v1)
    let v3 : UH0 = UH0_0(4, v2)
    let v4 : UH0 = UH0_0(2, v3)
    let v5 : UH0 = UH0_0(0, v4)
    let v6 : UH0 = UH0_1
    let v7 : UH0 = UH0_0(8, v6)
    let v8 : UH0 = UH0_0(6, v7)
    let v9 : UH0 = UH0_0(4, v8)
    let v10 : UH0 = UH0_0(2, v9)
    let v11 : UH0 = UH0_0(0, v10)
    let v12 : string = $"__expect / actual: %A{v5} / expected: %A{v11}"
    ()
method0()



## zip_with

In [ ]:
inl zip_with fn xs ys =
    inl rec loop acc xs ys =
        match xs, ys with
        | Cons (x, xs), Cons (y, ys) =>
            loop (fn x y :: acc) xs ys
        | _ => listm.rev acc
    loop [] xs ys

inl zip_with_ fn xs ys =
    let rec loop acc xs ys =
        match xs, ys with
        | Cons (x, xs), Cons (y, ys) =>
            loop (fn x y :: acc) xs ys
        | _ => listm.rev acc
    loop [] xs ys

()



In [ ]:
// // test

([ 1i32; 2; 3 ], [ 4; 5; 6 ])
||> zip_with (+)
|> _equal [ 5; 7; 9 ]

type UH0 =
    | UH0_0 of int32 * UH0
    | UH0_1
let rec method0 () : unit =
    let v0 : UH0 = UH0_1
    let v1 : UH0 = UH0_0(9, v0)
    let v2 : UH0 = UH0_0(7, v1)
    let v3 : UH0 = UH0_0(5, v2)
    let v4 : UH0 = UH0_1
    let v5 : UH0 = UH0_0(9, v4)
    let v6 : UH0 = UH0_0(7, v5)
    let v7 : UH0 = UH0_0(5, v6)
    let v8 : string = $"__expect / actual: %A{v3} / expected: %A{v7}"
    ()
method0()



## zip

In [ ]:
inl zip xs ys =
    zip_with pair xs ys

inl zip_ xs ys =
    zip_with_ pair xs ys

()



In [ ]:
// // test

([ 1i32; 2; 3 ], [ 4i32; 5; 6 ])
||> zip
|> _equal [ 1, 4; 2, 5; 3, 6 ]

type UH0 =
    | UH0_0 of int32 * int32 * UH0
    | UH0_1
let rec method0 () : unit =
    let v0 : UH0 = UH0_1
    let v1 : UH0 = UH0_0(3, 6, v0)
    let v2 : UH0 = UH0_0(2, 5, v1)
    let v3 : UH0 = UH0_0(1, 4, v2)
    let v4 : UH0 = UH0_1
    let v5 : UH0 = UH0_0(3, 6, v4)
    let v6 : UH0 = UH0_0(2, 5, v5)
    let v7 : UH0 = UH0_0(1, 4, v6)
    let v8 : string = $"__expect / actual: %A{v3} / expected: %A{v7}"
    ()
method0()



## indexed

In [ ]:
inl indexed list =
    (([], 0), list)
    ||> listm.fold fun (acc, i) x =>
        (i, x) :: acc, i + 1
    |> fst
    |> listm.rev

()



In [ ]:
// // test

listm.init 5i32 ((*) 2)
|> indexed
|> _equal [ 0i32, 0; 1, 2; 2, 4; 3, 6; 4, 8 ]

type UH0 =
    | UH0_0 of int32 * int32 * UH0
    | UH0_1
let rec method0 () : unit =
    let v0 : UH0 = UH0_1
    let v1 : UH0 = UH0_0(4, 8, v0)
    let v2 : UH0 = UH0_0(3, 6, v1)
    let v3 : UH0 = UH0_0(2, 4, v2)
    let v4 : UH0 = UH0_0(1, 2, v3)
    let v5 : UH0 = UH0_0(0, 0, v4)
    let v6 : UH0 = UH0_1
    let v7 : UH0 = UH0_0(4, 8, v6)
    let v8 : UH0 = UH0_0(3, 6, v7)
    let v9 : UH0 = UH0_0(2, 4, v8)
    let v10 : UH0 = UH0_0(1, 2, v9)
    let v11 : UH0 = UH0_0(0, 0, v10)
    let v12 : string = $"__expect / actual: %A{v5} / expected: %A{v11}"
    ()
method0()



## group_by

In [ ]:
inl group_by fn list =
    (list, [])
    ||> listm.foldBack fun x acc =>
        inl xk = fn x
        inl found, new_acc =
            ((false, []), acc)
            ||> listm.fold fun (found, acc') (k, xs) =>
                if k = xk
                then true, (k, x :: xs) :: acc'
                else found, (k, xs) :: acc'
        if found
        then new_acc
        else (xk, [ x ]) :: new_acc

()



In [ ]:
// // test

listm.init 10i32 id
|> group_by (fun x => x % 2 = 0)
|> _equal [ true, [ 0; 2; 4; 6; 8 ]; false, [ 1; 3; 5; 7; 9 ] ]

type UH0 =
    | UH0_0 of int32 * UH0
    | UH0_1
and UH1 =
    | UH1_0 of bool * UH0 * UH1
    | UH1_1
let rec method0 () : unit =
    let v0 : UH0 = UH0_1
    let v1 : UH0 = UH0_0(8, v0)
    let v2 : UH0 = UH0_0(6, v1)
    let v3 : UH0 = UH0_0(4, v2)
    let v4 : UH0 = UH0_0(2, v3)
    let v5 : UH0 = UH0_0(0, v4)
    let v6 : UH0 = UH0_1
    let v7 : UH0 = UH0_0(9, v6)
    let v8 : UH0 = UH0_0(7, v7)
    let v9 : UH0 = UH0_0(5, v8)
    let v10 : UH0 = UH0_0(3, v9)
    let v11 : UH0 = UH0_0(1, v10)
    let v12 : UH1 = UH1_1
    let v13 : UH1 = UH1_0(false, v11, v12)
    let v14 : UH1 = UH1_0(true, v5, v13)
    let v15 : UH0 = UH0_1
    let v16 : UH0 = UH0_0(8, v15)
    let v17 : UH0 = UH0_0(6, v16)
    let v18 : UH0 = UH0_0(4, v17)
    let v19 : UH0 = UH0_0(2, v18)
    let v20 : UH0 = UH0_0(0, v19)
    let v21 : UH0 = UH0_1
    let v22 : UH0 = UH0_0(9, v21)
    let v23 : UH0 = UH0_0(7, v22)
    let v24 : UH0 = UH0_0(5, v23)
    let v25 : UH0 = UH0_0(3, v24)
    let v26 : UH0 = UH0_0(1

## forall'

In [ ]:
inl forall' fn (head :: tail) =
    (true, tail)
    ||> listm.fold fun acc x =>
        acc && x = head

()



In [ ]:
// // test

[ 1i32; 1; 1; 1; 1 ]
|> forall' ((=) 1i32)
|> _equal true

let rec method0 () : unit =
    let v0 : string = $"__expect / actual: %A{true} / expected: %A{true}"
    ()
method0()



## last

In [ ]:
inl last list =
    list
    |> listm.rev
    |> item 0i32

()



In [ ]:
// // test

listm.init 10i32 id
|> last
|> _equal 9

let rec method0 () : unit =
    let v0 : string = $"__expect / actual: %A{9} / expected: %A{9}"
    ()
method0()



## try_pick

In [ ]:
inl try_pick fn list =
    inl rec body fn = function
        | [] => None
        | x :: xs =>
            match fn x with
            | Some y => Some y
            | None => loop xs
    and inl loop list =
        if var_is list |> not
        then body fn list
        else
            inl fn = join fn
            inl list = dyn list
            join body fn list
    loop list

()



In [ ]:
// // test

listm.init 10i32 id
|> try_pick (fun x => if x = 5i32 then Some x else None)
|> _equal (Some 5i32)

type [<Struct>] US0 =
    | US0_0
    | US0_1 of f1_0 : int32
let rec method0 () : unit =
    let v0 : US0 = US0_1(5)
    let v1 : US0 = US0_1(5)
    let v2 : string = $"__expect / actual: %A{v0} / expected: %A{v1}"
    ()
method0()

